<a href="https://colab.research.google.com/github/Anirudh-R-1201/Football_xG/blob/master/preprocessingStatsbomb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import requests
from pandas.io.json import json_normalize
import numpy as np
import pandas as pd
%reload_ext autoreload
%autoreload 2

In [8]:
ucl = 'https://raw.githubusercontent.com/statsbomb/open-data/master/data/matches/16/1.json'
wc = 'https://raw.githubusercontent.com/statsbomb/open-data/master/data/matches/43/3.json'

def get_json(url):
    r = requests.get(url)
    j = r.json()
    df = json_normalize(j, sep = "_")
    return df

ucl_df = get_json(ucl)
wc_df = get_json(wc)

matches = ucl_df.append(wc_df)
match_list = matches['match_id'].tolist()
len(match_list)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  import sys


65

In [9]:
master = pd.DataFrame()
for match in match_list:
    temp = get_json('https://raw.githubusercontent.com/statsbomb/open-data/master/data/events/%s.json' % match)
    temp['match_id'] = match
    master = master.append(temp)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  import sys


In [10]:
master.head

<bound method NDFrame.head of                                         id  ...  shot_follows_dribble
0     36dbfeaf-a586-4ae3-8c17-514a05dae661  ...                   NaN
1     7b4f4c0e-88ff-47ca-964c-0a54cb243ad2  ...                   NaN
2     8da6eaa4-7a2d-414e-b8a2-29b948b4a234  ...                   NaN
3     29ff277f-7c18-49d8-b365-d029b87a0c1d  ...                   NaN
4     2309b72d-cbd0-4b30-ad25-a3a12acd1e9a  ...                   NaN
...                                    ...  ...                   ...
3455  dc3b3b7d-eea8-4844-b5eb-d0836a3a001f  ...                   NaN
3456  49415967-323b-4d17-bc8a-7b670b411aef  ...                   NaN
3457  36c11393-a1c7-488c-8196-29b04ce7a0af  ...                   NaN
3458  f28357a8-62ae-4a6b-b7c6-bc3ce0ab3417  ...                   NaN
3459  82bcca63-8c9a-4d65-ad41-f7769ba291e3  ...                   NaN

[231376 rows x 133 columns]>

In [54]:
from google.colab import files 
master.to_csv('master.csv')
files.download("master.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000): 
        with pd.option_context("display.max_columns", 1000): 
            display(df.tail().transpose())

display_all(master)

,3455,3456,3457,3458,3459
id,dc3b3b7d-eea8-4844-b5eb-d0836a3a001f,49415967-323b-4d17-bc8a-7b670b411aef,36c11393-a1c7-488c-8196-29b04ce7a0af,f28357a8-62ae-4a6b-b7c6-bc3ce0ab3417,82bcca63-8c9a-4d65-ad41-f7769ba291e3
index,3456,3457,3458,3459,3460
period,2,2,2,2,2
timestamp,00:51:15.533,00:51:15.533,00:52:07.813,00:52:09.280,00:52:09.280
minute,96,96,97,97,97
second,15,15,7,9,9
possession,165,165,166,166,166
duration,NaN,0,1.467,NaN,NaN
type_id,22,21,30,34,34
type_name,Foul Committed,Foul Won,Pass,Half End,Half End


In [13]:
shots = master[master['type_id'] == 16]
passes = master[master['type_id'] == 30]

In [14]:
chains = pd.DataFrame()
for item in list(shots['possession'].unique()):
    temp = master[master['possession'] == item]
    chains = chains.append(temp)
chains = chains[chains['type_id'] != 17]
chains = chains[chains['type_id'] != 42]
chains = chains.reset_index().drop('level_0', axis=1)

chains['previous_event'] = chains['type_name'].shift()
chains['previous_event'] = chains['previous_event'].str.lower().str.replace(' ', '_').str.replace('*', '')
chains['previous_event'] = np.where(chains['previous_event'] == 'goal_keeper', 'goalkeeper', chains['previous_event'])

In [23]:
chains.dtypes

id                      object
index                    int64
period                   int64
timestamp               object
minute                   int64
                         ...  
follows_error            int64
follows_block            int64
follows_dispossessed     int64
follows_50/50            int64
follows_dribble          int64
Length: 146, dtype: object

In [16]:
important_pre_shot = ['pass', 'ball_recovery', 'duel', 'interception', 'clearance', 'goalkeeper', 'miscontrol',
                      'error', 'block', 'dispossessed', '50/50', 'dribble']

for event in important_pre_shot:
    chains['follows_%s' % event] = np.where(chains['previous_event'] == event, 1, 0)

In [17]:
chains_filt = chains[['match_id', 'possession', 'duration']]
chains_filt.columns = ['match_id', 'possession', 'chain_duration']
chains_dur = chains_filt.pivot_table(index=['match_id', 'possession'], values='chain_duration', aggfunc='sum')
chains_dur.reset_index(inplace=True)

In [18]:
assisted_shots = shots[shots['shot_key_pass_id'].notna()]
assist_passes = passes[passes['pass_assisted_shot_id'].notna()]
assist_passes['pass_body_part_name'] = np.where((assist_passes['pass_body_part_name'] == 'Right Foot')
                                 | (assist_passes['pass_body_part_name'] == 'Left Foot'), 'foot',
                                np.where(assist_passes['pass_body_part_name'] == 'Head', 'head', 'other'))
pass_column = [i for i in assist_passes.columns if i.startswith('pass')]
assist_passes = assist_passes[pass_column]
assisted_shots = assisted_shots.drop(pass_column, axis=1)
shot_pass = assisted_shots.merge(assist_passes, left_on='id', right_on='pass_assisted_shot_id', how='left')
other_shots = shots[shots['shot_key_pass_id'].isna()]
shots = other_shots.append(shot_pass)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [19]:
shots['location'] = shots['location'].astype(str).str.strip('[]')
shots['x_loc'] = shots['location'].str.split(', ').str.get(0).str.strip().astype(float)
shots['y_loc'] = shots['location'].str.split(', ').str.get(1).str.strip().astype(float)

In [25]:
print(chains_dur)

       match_id  possession  chain_duration
0          7525           2       21.081000
1          7525           3        1.772000
2          7525           4        5.760000
3          7525           5       16.420000
4          7525           6        6.000000
...         ...         ...             ...
11416     18245         159        5.161710
11417     18245         160       13.592516
11418     18245         161        2.705703
11419     18245         162       16.057498
11420     18245         163        7.711094

[11421 rows x 3 columns]


In [26]:
shots = shots[shots['shot_type_name'] != 'Penalty']

shots = shots.reset_index().drop('level_0', axis=1)
shots['goal'] = np.where(shots['shot_outcome_name'] == 'Goal', 1, 0)
shots['assisted'] = np.where(shots['shot_key_pass_id'].isna(), 0, 1)
shots['x_dist'] = 120 - shots['x_loc']
shots['y_dist'] = abs(40 - shots['y_loc'])
shots['distance'] = np.sqrt((shots['x_dist']**2 + shots['y_dist']**2))
shots['angle'] = np.arctan((shots['y_dist'] / shots['x_dist']))

shots['shot_body_part_name'] = np.where((shots['shot_body_part_name'] == 'Right Foot')
                                 | (shots['shot_body_part_name'] == 'Left Foot'), 'foot',
                                np.where(shots['shot_body_part_name'] == 'Head', 'head', 'other'))

shots['build_up_duration'] = chains_dur['chain_duration'] - shots['duration']
shots['direct_free_kick'] = np.where((shots['play_pattern_name'] == 'From Free Kick')
                                      & (shots['pass_assisted_shot_id'].isna()), 1, 0)
shots['pass_angle'] = np.where(shots['pass_angle'] < 0, shots['pass_angle'] * -1, shots['pass_angle'])

In [31]:
from itertools import chain
from ast import literal_eval

freeze_frame = shots[['shot_freeze_frame', 'id', 'x_loc', 'y_loc']]
freeze_frame_df = pd.DataFrame(freeze_frame)
#freeze_frame_df['shot_freeze_frame'] = freeze_frame_df['shot_freeze_frame'].apply(literal_eval)
#freeze_frame_df['shot_freeze_frame'] = freeze_frame_df['shot_freeze_frame'].to_json()
#freeze_frame_df['shot_freeze_frame'] = literal_eval(json.dumps(freeze_frame_df['shot_freeze_frame'], ensure_ascii=False).encode('utf8'))
def add_shot_id(row):
    return [item.update({'shot_id': row['id'], 'shot_x_loc': row['x_loc'], 'shot_y_loc': row['y_loc']}) 
            for item in row['shot_freeze_frame']]

freeze_frame_df.apply(add_shot_id, axis=1)
freeze_frame_df.head()

,shot_freeze_frame,id,x_loc,y_loc
0,"[{'location': [102.9, 48.6], 'player': {'id': ...",e892be83-4599-4b42-9b73-4a85cf0f6a49,94.5,26.9
1,"[{'location': [87.4, 47.1], 'player': {'id': 3...",8b10cb3f-c124-468d-acc6-d1bd1a9c5c04,101.2,30.6
2,"[{'location': [102.9, 32.6], 'player': {'id': ...",775fddd2-527e-4a62-97fd-fcce8c6581b5,103.7,35.9
3,"[{'location': [101.7, 31.6], 'player': {'id': ...",d44d0170-edc7-42a2-98a1-0c00a3780689,106.4,55.3
4,"[{'location': [112.2, 18.0], 'player': {'id': ...",4fdb4cd6-fa1b-4524-a798-a30cf5143734,108.8,32.4


In [32]:
chained = chain.from_iterable(freeze_frame_df['shot_freeze_frame'])
ff_expanded = pd.DataFrame(list(chained))
ff_expanded.head()

,location,player,position,teammate,shot_id,shot_x_loc,shot_y_loc
0,"[102.9, 48.6]","{'id': 5207, 'name': 'Cristiano Ronaldo dos Sa...","{'id': 24, 'name': 'Left Center Forward'}",True,e892be83-4599-4b42-9b73-4a85cf0f6a49,94.5,26.9
1,"[102.8, 55.0]","{'id': 19677, 'name': 'Karim Benzema'}","{'id': 22, 'name': 'Right Center Forward'}",True,e892be83-4599-4b42-9b73-4a85cf0f6a49,94.5,26.9
2,"[117.7, 39.1]","{'id': 3630, 'name': 'Loris Karius'}","{'id': 1, 'name': 'Goalkeeper'}",False,e892be83-4599-4b42-9b73-4a85cf0f6a49,94.5,26.9
3,"[102.6, 22.4]","{'id': 3664, 'name': 'Trent Alexander-Arnold'}","{'id': 2, 'name': 'Right Back'}",False,e892be83-4599-4b42-9b73-4a85cf0f6a49,94.5,26.9
4,"[100.9, 31.5]","{'id': 3471, 'name': 'Dejan Lovren'}","{'id': 3, 'name': 'Right Center Back'}",False,e892be83-4599-4b42-9b73-4a85cf0f6a49,94.5,26.9


In [33]:
ff_expanded['location'] = ff_expanded['location'].astype(str).str.strip('[]')
ff_expanded['x_loc'] = ff_expanded['location'].str.split(', ').str.get(0).str.strip().astype(float)
ff_expanded['y_loc'] = ff_expanded['location'].str.split(', ').str.get(1).str.strip().astype(float)

In [34]:
ff_expanded['player_x_dist'] = abs(ff_expanded['shot_x_loc'] - ff_expanded['x_loc'])
ff_expanded['player_y_dist'] = abs(ff_expanded['shot_y_loc'] - ff_expanded['y_loc'])
ff_expanded['player_distance_from_shooter'] = np.where(ff_expanded['player_x_dist'] < 0,
                                                    -1*np.sqrt((ff_expanded['player_x_dist']**2 + ff_expanded['player_y_dist']**2)),
                                                           np.sqrt((ff_expanded['player_x_dist']**2 + ff_expanded['player_y_dist']**2)))
ff_expanded['player_angle_to_shooter'] = np.arctan((ff_expanded['player_y_dist'] / ff_expanded['player_x_dist']))

In [35]:
def area(x1, y1, x2, y2, x3, y3):
    return abs((x1 * (y2 - y3) + x2 * (y3 - y1) + x3 * (y1 - y2)) / 2.0)

def get_shot_cone(row):
    return area(row['shot_x_loc'], row['shot_x_loc'], 120, 44, 120, 36)

ff_expanded['shot_cone'] = ff_expanded.apply(get_shot_cone, axis=1)

def in_shot_cone(x1, y1, x2, y2, x3, y3, x, y): 
    A = area (x1, y1, x2, y2, x3, y3) 
    A1 = area (x, y, x2, y2, x3, y3)  
    A2 = area (x1, y1, x, y, x3, y3)   
    A3 = area (x1, y1, x2, y2, x, y) 
    if(A == A1 + A2 + A3): 
        return 1
    else: 
        return 0

def player_in_cone(row):
    return in_shot_cone(row['shot_x_loc'], row['shot_x_loc'], 120, 44, 120, 36, row['x_loc'], row['y_loc'])

ff_expanded['player_in_shot_cone'] = ff_expanded.apply(player_in_cone, axis=1)

ff_expanded['player_in_shot_cone'] = np.where(ff_expanded['player_angle_to_shooter'].isna(), 
                                              1, ff_expanded['player_in_shot_cone'])

ff_expanded['player_angle_to_shooter'] = ff_expanded['player_angle_to_shooter'].fillna(0)

In [36]:
shot_cone = pd.pivot_table(ff_expanded, index=['shot_id'], values=['shot_cone']).reset_index()
shot_cone.columns = ['id', 'shot_cone']
players_in_cone = pd.pivot_table(ff_expanded, index=['shot_id'], values=['player_in_shot_cone'], aggfunc=np.sum).reset_index()
players_in_cone.columns = ['id', 'num_players_in_shot_cone']
min_distance = pd.pivot_table(ff_expanded[ff_expanded['teammate'] == True], index=['shot_id'],
                                          values=['player_distance_from_shooter'], aggfunc=np.min).reset_index()
min_distance.columns = ['id', 'distance_to_nearest_opponent']
shots = shots.merge(shot_cone, on='id', how='left')
shots = shots.merge(players_in_cone, on='id', how='left')
shots = shots.merge(min_distance, on='id', how='left')

In [47]:
feature_cols = ['duration', 'follows_50/50', 'follows_ball_recovery', 'follows_block', 
                'follows_clearance', 'follows_dispossessed', 'follows_dribble', 'follows_duel', 'follows_error', 
                'follows_goalkeeper', 'follows_interception', 'follows_miscontrol', 'follows_pass', 
                'pass_aerial_won', 'pass_angle', 'pass_body_part_name', 
                'pass_cross', 'pass_cut_back', 'pass_deflected', 'pass_height_name', 'pass_length', 
                'pass_switch', 'pass_through_ball', 'play_pattern_name', 'shot_aerial_won', 
                'shot_body_part_name', 'shot_first_time', 'shot_one_on_one', 'shot_open_goal', 
                'shot_technique_name', 'under_pressure', 'goal', 'distance', 'angle', 
                'build_up_duration', 'direct_free_kick', 'shot_cone', 'num_players_in_shot_cone', 
                'distance_to_nearest_opponent']

shots_final = pd.DataFrame()#shots[feature_cols]
for i in feature_cols:
  try:
    #print(shots[i],i)
    shots_final[i] = shots[i]
  except:
    pass

In [48]:
shots_final

,duration,pass_aerial_won,pass_angle,pass_body_part_name,pass_cross,pass_cut_back,pass_deflected,pass_height_name,pass_length,pass_switch,pass_through_ball,play_pattern_name,shot_aerial_won,shot_body_part_name,shot_first_time,shot_one_on_one,shot_open_goal,shot_technique_name,under_pressure,goal,distance,angle,build_up_duration,direct_free_kick,shot_cone,num_players_in_shot_cone,distance_to_nearest_opponent
0,1.301567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,From Free Kick,NaN,other,NaN,NaN,NaN,Normal,NaN,0,28.668101,0.474568,19.779433,1,102.0,0,16.890826
1,0.054247,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,From Free Kick,NaN,other,True,NaN,NaN,Normal,NaN,0,21.019039,0.463648,1.717753,1,75.2,0,10.439349
2,0.160234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,From Free Kick,NaN,other,NaN,NaN,NaN,Normal,True,0,16.807736,0.246422,5.599766,1,65.2,0,3.395585
3,0.383183,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Regular Play,NaN,other,NaN,NaN,NaN,Normal,NaN,0,20.470711,0.844154,16.036817,0,54.4,0,7.731753
4,0.280194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,From Throw In,NaN,other,NaN,NaN,NaN,Normal,NaN,0,13.535139,0.596199,5.719806,0,44.8,0,3.104835
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1661,1.293000,NaN,1.119343,foot,True,NaN,NaN,High Pass,36.674240,NaN,NaN,From Free Kick,True,other,NaN,NaN,NaN,Normal,True,0,11.180340,0.463648,30.147000,0,40.0,0,3.162278
1662,0.400000,NaN,1.272297,foot,True,NaN,NaN,High Pass,27.202942,NaN,NaN,Regular Play,NaN,other,NaN,NaN,NaN,Normal,NaN,0,9.000000,0.000000,32.494000,0,36.0,0,3.162278
1663,0.920000,NaN,1.618380,foot,NaN,NaN,NaN,Ground Pass,21.023796,NaN,NaN,From Corner,NaN,other,NaN,NaN,NaN,Normal,NaN,0,19.924859,0.305879,3.373000,0,76.0,1,9.055385
1664,0.973000,NaN,0.785398,head,NaN,NaN,NaN,Low Pass,22.627417,NaN,NaN,Regular Play,NaN,other,True,NaN,NaN,Normal,True,0,23.021729,0.970746,37.440000,0,52.0,1,9.486833


In [49]:
print(shots_final.isnull().sum())

duration                           0
pass_aerial_won                 1648
pass_angle                       468
pass_body_part_name              468
pass_cross                      1349
pass_cut_back                   1640
pass_deflected                  1663
pass_height_name                 468
pass_length                      468
pass_switch                     1566
pass_through_ball               1651
play_pattern_name                  0
shot_aerial_won                 1540
shot_body_part_name                0
shot_first_time                 1301
shot_one_on_one                 1609
shot_open_goal                  1649
shot_technique_name                0
under_pressure                  1332
goal                               0
distance                           0
angle                              0
build_up_duration                  0
direct_free_kick                   0
shot_cone                          0
num_players_in_shot_cone           0
distance_to_nearest_opponent      16
d

In [52]:
bool_cols = ['pass_aerial_won', 'pass_cross', 'pass_cut_back', 'pass_deflected',
             'pass_switch', 'pass_through_ball', 'shot_aerial_won', 'shot_first_time', 'shot_one_on_one',
             'shot_open_goal', 'under_pressure']

for col in bool_cols:
    shots_final[col] = np.where(shots_final[col].isna(), 0, 1)
    
fix_numeric = ['pass_angle', 'pass_length', 'distance_to_nearest_opponent']
for col in fix_numeric:
    shots_final[col+'_na'] = pd.isnull(shots_final[col])

shots_final['distance_to_nearest_opponent'] = shots_final['distance_to_nearest_opponent'].fillna(shots_final['distance_to_nearest_opponent'].max())

shots_final['pass_length'] = shots_final['pass_length'].fillna(0)
shots_final['pass_angle'] = shots_final['pass_angle'].fillna(0)
shots_final

,duration,pass_aerial_won,pass_angle,pass_body_part_name,pass_cross,pass_cut_back,pass_deflected,pass_height_name,pass_length,pass_switch,pass_through_ball,play_pattern_name,shot_aerial_won,shot_body_part_name,shot_first_time,shot_one_on_one,shot_open_goal,shot_technique_name,under_pressure,goal,distance,angle,build_up_duration,direct_free_kick,shot_cone,num_players_in_shot_cone,distance_to_nearest_opponent,pass_angle_na,pass_length_na,distance_to_nearest_opponent_na
0,1.301567,1,0.000000,NaN,1,1,1,NaN,0.000000,1,1,From Free Kick,1,other,1,1,1,Normal,1,0,28.668101,0.474568,19.779433,1,102.0,0,16.890826,False,False,False
1,0.054247,1,0.000000,NaN,1,1,1,NaN,0.000000,1,1,From Free Kick,1,other,1,1,1,Normal,1,0,21.019039,0.463648,1.717753,1,75.2,0,10.439349,False,False,False
2,0.160234,1,0.000000,NaN,1,1,1,NaN,0.000000,1,1,From Free Kick,1,other,1,1,1,Normal,1,0,16.807736,0.246422,5.599766,1,65.2,0,3.395585,False,False,False
3,0.383183,1,0.000000,NaN,1,1,1,NaN,0.000000,1,1,Regular Play,1,other,1,1,1,Normal,1,0,20.470711,0.844154,16.036817,0,54.4,0,7.731753,False,False,False
4,0.280194,1,0.000000,NaN,1,1,1,NaN,0.000000,1,1,From Throw In,1,other,1,1,1,Normal,1,0,13.535139,0.596199,5.719806,0,44.8,0,3.104835,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1661,1.293000,1,1.119343,foot,1,1,1,High Pass,36.674240,1,1,From Free Kick,1,other,1,1,1,Normal,1,0,11.180340,0.463648,30.147000,0,40.0,0,3.162278,False,False,False
1662,0.400000,1,1.272297,foot,1,1,1,High Pass,27.202942,1,1,Regular Play,1,other,1,1,1,Normal,1,0,9.000000,0.000000,32.494000,0,36.0,0,3.162278,False,False,False
1663,0.920000,1,1.618380,foot,1,1,1,Ground Pass,21.023796,1,1,From Corner,1,other,1,1,1,Normal,1,0,19.924859,0.305879,3.373000,0,76.0,1,9.055385,False,False,False
1664,0.973000,1,0.785398,head,1,1,1,Low Pass,22.627417,1,1,Regular Play,1,other,1,1,1,Normal,1,0,23.021729,0.970746,37.440000,0,52.0,1,9.486833,False,False,False


In [51]:
shots_final.to_csv('shots.csv', index=False)
files.download("shots.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>